# Airfoil's Aerodynamic Coefficient Prediction using ANNs

In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [2]:
# importing the dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow import keras
from keras import Input
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
from keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt

In [3]:
# reading the csv file
df4 = pd.read_csv("/home/nevilcp/ML_Aero/results/NACA4D_10/NACA4D_10.csv")
df5 = pd.read_csv("/home/nevilcp/ML_Aero/results/NACA5D_10/NACA5D_10.csv")

In [4]:
# printing the first 10 rows of the dataset of NACA4D_10
df4.head(10)

,t,m,p,yU1,yU2,yU3,yU4,yU5,yU6,yU7,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
0,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-10.0,0.2,100000,-1.158,0.02657,0.005
1,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-9.0,0.2,100000,-1.043,0.02194,0.004
2,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-8.0,0.2,100000,-0.928,0.01876,0.004
3,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-7.0,0.2,100000,-0.813,0.01463,0.004
4,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-6.0,0.2,100000,-0.697,0.02806,0.003
5,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-5.0,0.2,100000,-0.581,0.01660,0.003
6,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-4.0,0.2,100000,-0.465,0.01146,0.002
7,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-3.0,0.2,100000,-0.349,0.00893,0.002
8,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-2.0,0.2,100000,-0.233,0.01393,0.001
9,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-1.0,0.2,100000,-0.116,0.01358,0.001


In [5]:
# printing the first 10 rows of the dataset of NACA5D_10
df5.head(10)

,t,CL_design,p,s,yU1,yU2,yU3,yU4,yU5,yU6,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
0,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-10.0,0.1,100000,-1.140,0.02494,0.005
1,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-9.0,0.1,100000,-1.027,0.02116,0.004
2,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-8.0,0.1,100000,-0.914,0.01697,0.004
3,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-7.0,0.1,100000,-0.800,0.01364,0.003
4,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-6.0,0.1,100000,-0.687,0.02483,0.003
5,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-5.0,0.1,100000,-0.572,0.01565,0.002
6,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-4.0,0.1,100000,-0.458,0.01114,0.002
7,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-3.0,0.1,100000,-0.344,0.00882,0.001
8,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-2.0,0.1,100000,-0.229,0.01391,0.001
9,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-1.0,0.1,100000,-0.115,0.01357,0.000


In [6]:
# checking for missing values in the NACA4D_10 dataset
df4.isnull().sum()

t        0
m        0
p        0
yU1      0
yU2      0
yU3      0
yU4      0
yU5      0
yU6      0
yU7      0
yU8      0
yU9      0
yU10     0
yL1      0
yL2      0
yL3      0
yL4      0
yL5      0
yL6      0
yL7      0
yL8      0
yL9      0
yL10     0
alpha    0
M        0
Re       0
CL       0
CD       0
Cm       0
dtype: int64

In [7]:
# checking for missing values in the NACA5D_10 dataset
df5.isnull().sum()

t            0
CL_design    0
p            0
s            0
yU1          0
yU2          0
yU3          0
yU4          0
yU5          0
yU6          0
yU7          0
yU8          0
yU9          0
yU10         0
yL1          0
yL2          0
yL3          0
yL4          0
yL5          0
yL6          0
yL7          0
yL8          0
yL9          0
yL10         0
alpha        0
M            0
Re           0
CL           0
CD           0
Cm           0
dtype: int64

In [8]:
# description of the NACA4D_10 dataset
df4.describe()

,t,m,p,yU1,yU2,yU3,yU4,yU5,yU6,yU7,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
count,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,...,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.00000,176400.000000,176400.000000,176400.000000,176400.000000
mean,20.000000,4.500000,40.000000,0.056883,0.096159,0.125194,0.138691,0.136083,0.120107,0.094865,...,-0.025367,-0.013472,-0.006526,-0.003090,0.000000,0.20000,300000.000000,0.692283,0.026646,-0.134151
std,10.000028,2.872289,22.912943,0.030652,0.043042,0.052297,0.056240,0.054535,0.048225,0.039002,...,0.037872,0.026573,0.014169,0.004346,6.055318,0.08165,141421.757093,0.926925,0.010960,0.113007
min,5.000000,0.000000,5.000000,0.009825,0.017881,0.023099,0.024990,0.023669,0.019965,0.014949,...,-0.104640,-0.067474,-0.034638,-0.011856,-10.000000,0.10000,100000.000000,-1.489000,0.001800,-0.514000
25%,10.000000,2.000000,22.500000,0.031423,0.060652,0.084089,0.094866,0.094677,0.083039,0.065317,...,-0.053389,-0.032844,-0.016811,-0.006339,-5.000000,0.10000,200000.000000,-0.011000,0.018990,-0.199000
50%,20.000000,4.500000,40.000000,0.054270,0.096111,0.124907,0.138240,0.136379,0.119788,0.094447,...,-0.025009,-0.014037,-0.006982,-0.003241,-0.000000,0.20000,300000.000000,0.691000,0.025390,-0.106000
75%,30.000000,7.000000,57.500000,0.078524,0.129458,0.166614,0.182079,0.176607,0.156303,0.123175,...,0.001474,0.004181,0.002635,-0.000235,5.000000,0.30000,400000.000000,1.379000,0.032540,-0.046000
max,35.000000,9.000000,75.000000,0.149810,0.208025,0.246146,0.264431,0.256034,0.230429,0.194578,...,0.074748,0.071348,0.042540,0.011816,10.000000,0.30000,500000.000000,3.599000,0.235470,0.025000


In [9]:
# description of the NACA5D_10 dataset
df5.describe()

,t,CL_design,p,s,yU1,yU2,yU3,yU4,yU5,yU6,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
count,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,...,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000
mean,19.964508,0.841171,27.648625,0.493345,0.061993,0.102135,0.127800,0.134156,0.122524,0.098694,...,-0.050760,-0.035252,-0.019158,-0.006666,0.000000,0.200000,300000.000000,0.219104,0.025913,0.008665
std,10.002176,0.595675,16.859596,0.499957,0.035497,0.050154,0.062040,0.067278,0.064190,0.054114,...,0.038246,0.024719,0.012716,0.004107,6.055314,0.081686,141421.655012,0.926011,0.011283,0.092562
min,5.000000,0.000000,5.000000,0.000000,0.002846,-0.006454,-0.022993,-0.039093,-0.048037,-0.046811,...,-0.171128,-0.127512,-0.072044,-0.023250,-10.000000,0.100000,100000.000000,-2.607000,0.002690,-0.169000
25%,10.000000,0.200000,15.000000,0.000000,0.033635,0.064809,0.081379,0.082846,0.074521,0.059894,...,-0.078123,-0.052613,-0.027989,-0.009654,-5.000000,0.100000,200000.000000,-0.485000,0.018260,-0.031000
50%,20.000000,0.800000,25.000000,0.000000,0.057851,0.101474,0.127721,0.133115,0.120174,0.099175,...,-0.050658,-0.034769,-0.018871,-0.006613,-0.000000,0.200000,300000.000000,0.218000,0.024650,-0.004000
75%,30.000000,1.400000,45.000000,1.000000,0.085249,0.134294,0.169672,0.179965,0.165684,0.137987,...,-0.022239,-0.016698,-0.009816,-0.003398,5.000000,0.300000,400000.000000,0.928000,0.031580,0.018000
max,35.000000,1.800000,75.000000,1.000000,0.165549,0.250220,0.311403,0.334353,0.308679,0.262679,...,0.048283,0.022775,0.009953,0.002097,10.000000,0.300000,500000.000000,3.020000,0.617020,0.453000


### Dropping columns

In [10]:
# dropping columns t, m & p in the NACA4D_10 dataset
df4 = df4.drop(columns=['t', 'm', 'p'], axis=1)
df4.head(10)

,yU1,yU2,yU3,yU4,yU5,yU6,yU7,yU8,yU9,yU10,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-10.0,0.2,100000,-1.158,0.02657,0.005
1,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-9.0,0.2,100000,-1.043,0.02194,0.004
2,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-8.0,0.2,100000,-0.928,0.01876,0.004
3,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-7.0,0.2,100000,-0.813,0.01463,0.004
4,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-6.0,0.2,100000,-0.697,0.02806,0.003
5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-5.0,0.2,100000,-0.581,0.01660,0.003
6,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-4.0,0.2,100000,-0.465,0.01146,0.002
7,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-3.0,0.2,100000,-0.349,0.00893,0.002
8,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-2.0,0.2,100000,-0.233,0.01393,0.001
9,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-1.0,0.2,100000,-0.116,0.01358,0.001


In [11]:
# dropping columns t, CL_design, p & s in the NACA5D_10 dataset
df5 = df5.drop(columns=['t', 'CL_design', 'p', 's'], axis=1)
df5.head(10)

,yU1,yU2,yU3,yU4,yU5,yU6,yU7,yU8,yU9,yU10,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-10.0,0.1,100000,-1.140,0.02494,0.005
1,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-9.0,0.1,100000,-1.027,0.02116,0.004
2,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-8.0,0.1,100000,-0.914,0.01697,0.004
3,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-7.0,0.1,100000,-0.800,0.01364,0.003
4,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-6.0,0.1,100000,-0.687,0.02483,0.003
5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-5.0,0.1,100000,-0.572,0.01565,0.002
6,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-4.0,0.1,100000,-0.458,0.01114,0.002
7,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-3.0,0.1,100000,-0.344,0.00882,0.001
8,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-2.0,0.1,100000,-0.229,0.01391,0.001
9,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-1.0,0.1,100000,-0.115,0.01357,0.000


In [12]:
# shuffling all the rows for both the datasets
df4 = df4.sample(frac=1)

df5 = df5.sample(frac=1)

In [13]:
# defining the feature and target columns for both the datasets
X4 = df4.drop(columns=['CL', 'CD', 'Cm']) 
y4 = df4[['CL', 'CD', 'Cm']]

X5 = df5.drop(columns=['CL', 'CD', 'Cm'])
y5 = df5[['CL', 'CD', 'Cm']]

In [14]:
# displaying the features table for NACA4D_10
X4.head(10)

,yU1,yU2,yU3,yU4,yU5,yU6,yU7,yU8,yU9,yU10,...,yL4,yL5,yL6,yL7,yL8,yL9,yL10,alpha,M,Re
153622,0.070474,0.121125,0.162647,0.188220,0.194052,0.179926,0.147209,0.101278,0.053117,0.016327,...,-0.063625,-0.043447,-0.019881,-0.003249,0.002666,0.001719,-0.001209,-3.0,0.3,100000
116791,0.098796,0.159528,0.204827,0.227930,0.225472,0.198813,0.155181,0.103421,0.053570,0.017081,...,-0.122897,-0.106509,-0.081532,-0.055410,-0.033234,-0.016830,-0.006745,-0.0,0.3,200000
41203,0.060160,0.104077,0.134470,0.144887,0.137388,0.116598,0.087853,0.056861,0.029071,0.009631,...,-0.105018,-0.099394,-0.083190,-0.061767,-0.039632,-0.020439,-0.007264,-9.0,0.3,300000
98362,0.051364,0.092616,0.125604,0.144003,0.144479,0.127917,0.099821,0.066179,0.033976,0.010626,...,-0.056419,-0.044954,-0.031982,-0.020166,-0.011510,-0.005994,-0.002989,9.0,0.1,400000
118548,0.079699,0.133853,0.174939,0.197332,0.198585,0.179957,0.144592,0.098383,0.051497,0.016239,...,-0.103719,-0.086068,-0.059751,-0.035372,-0.018745,-0.009089,-0.004339,-7.0,0.3,100000
169306,0.093709,0.153491,0.201626,0.229564,0.231721,0.207789,0.164563,0.110731,0.057442,0.017774,...,-0.072635,-0.053127,-0.033039,-0.017245,-0.008124,-0.004265,-0.003148,-6.0,0.3,300000
167723,0.019356,0.048215,0.079742,0.104102,0.113383,0.105634,0.085385,0.057528,0.029252,0.008247,...,0.053692,0.066036,0.065597,0.055174,0.037775,0.019001,0.004714,7.0,0.2,200000
117121,0.014868,0.034519,0.055203,0.071917,0.080731,0.079782,0.067644,0.046979,0.024378,0.007005,...,0.021695,0.033334,0.039853,0.037641,0.027448,0.014267,0.003524,-6.0,0.1,300000
64240,0.012288,0.026125,0.039085,0.048422,0.052192,0.049873,0.041284,0.028281,0.014641,0.004344,...,-0.001618,0.004839,0.009945,0.011362,0.008942,0.004694,0.000937,-9.0,0.3,500000
31311,0.020543,0.038225,0.050885,0.056956,0.056160,0.049766,0.039635,0.026753,0.013963,0.004526,...,-0.043019,-0.038520,-0.030089,-0.020154,-0.011811,-0.005837,-0.002246,-10.0,0.2,200000


In [15]:
# displaying the features table for NACA5D_10
X5.head(10)

,yU1,yU2,yU3,yU4,yU5,yU6,yU7,yU8,yU9,yU10,...,yL4,yL5,yL6,yL7,yL8,yL9,yL10,alpha,M,Re
167495,0.035663,0.070633,0.089233,0.091679,0.082101,0.066400,0.048894,0.031831,0.016932,0.006379,...,-0.158424,-0.154687,-0.133541,-0.101017,-0.064946,-0.032710,-0.010521,10.0,0.2,100000
172100,0.104335,0.142850,0.178323,0.189334,0.176977,0.147631,0.109470,0.069922,0.035381,0.011642,...,-0.110089,-0.110907,-0.096818,-0.074094,-0.048620,-0.025399,-0.008990,-5.0,0.1,100000
226267,0.065372,0.140502,0.196714,0.208387,0.172753,0.117283,0.066269,0.029798,0.010119,0.002628,...,0.107540,0.072709,0.033226,0.004336,-0.009315,-0.009760,-0.004153,3.0,0.3,500000
209355,0.097038,0.163656,0.217993,0.246767,0.241481,0.202263,0.140910,0.076885,0.030768,0.008470,...,-0.004137,0.002730,-0.004219,-0.016597,-0.024023,-0.019789,-0.008562,-4.0,0.1,500000
235769,0.053904,0.095650,0.124460,0.136286,0.130977,0.111990,0.084856,0.055461,0.029269,0.011090,...,-0.113651,-0.105711,-0.087653,-0.064636,-0.040935,-0.020199,-0.005811,-8.0,0.2,300000
109414,0.041260,0.078462,0.094760,0.092041,0.081434,0.065637,0.047467,0.029680,0.014690,0.004612,...,-0.008096,-0.013486,-0.014465,-0.012516,-0.008998,-0.005158,-0.002171,-6.0,0.2,100000
235814,0.053904,0.095650,0.124460,0.136286,0.130977,0.111990,0.084856,0.055461,0.029269,0.011090,...,-0.113651,-0.105711,-0.087653,-0.064636,-0.040935,-0.020199,-0.005811,-5.0,0.2,500000
25534,0.049081,0.089342,0.115401,0.124827,0.118198,0.099668,0.074595,0.048067,0.024635,0.008379,...,-0.125075,-0.118494,-0.099979,-0.074890,-0.048324,-0.024848,-0.008558,9.0,0.1,100000
58879,0.036813,0.072012,0.090801,0.092996,0.082586,0.065903,0.048060,0.031807,0.018575,0.009694,...,-0.157156,-0.154187,-0.133950,-0.101808,-0.065036,-0.031140,-0.007219,6.0,0.3,400000
26802,0.019650,0.035762,0.046198,0.049980,0.047338,0.039929,0.029897,0.019278,0.009897,0.003388,...,-0.049980,-0.047338,-0.039929,-0.029897,-0.019278,-0.009897,-0.003388,-4.0,0.3,200000


In [16]:
# displaying the targets table for NACA4D_10
y4.head(10)

,CL,CD,Cm
153622,0.991,0.03251,-0.303
116791,1.070,0.03779,-0.195
41203,-0.969,0.02863,-0.024
98362,1.888,0.03338,-0.166
118548,0.100,0.03382,-0.218
169306,0.645,0.03048,-0.278
167723,1.953,0.02911,-0.273
117121,0.150,0.01832,-0.206
64240,-0.635,0.01828,-0.105
31311,-1.037,0.02164,-0.035


In [17]:
# displaying the targets table for NACA5D_10
y5.head(10)

,CL,CD,Cm
167495,0.862,0.03265,0.070
172100,-0.247,0.02068,-0.002
226267,1.127,0.02623,0.011
209355,0.419,0.02355,-0.087
235769,-0.932,0.02733,-0.016
109414,-0.355,0.02227,-0.029
235814,-0.526,0.02345,-0.021
25534,1.198,0.03424,-0.016
58879,0.219,0.02414,0.104
26802,-0.498,0.01630,0.003


## Splitting the Dataset

In [18]:
# assigning X_train, y_train, X_test & y_test
X_train, X_val, y_train, y_val = train_test_split(X4, y4, train_size=0.8, random_state=42)
X_test, y_test = X5, y5

In [19]:
# normalizing the training, validation & testing datasets
X_scaler = StandardScaler()
X_train = X_scaler.fit_transform(X_train)
X_val = X_scaler.transform(X_val)
X_test = X_scaler.transform(X_test)

In [20]:
# displaying X_train, X_val & X_test after standardization
display(pd.DataFrame(X_train).head())
display(pd.DataFrame(X_val).head())
display(pd.DataFrame(X_test).head())

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,-0.894234,-0.987075,-1.067565,-1.131536,-1.191841,-1.247358,-1.281640,-1.265464,-1.216370,-1.137092,...,-0.238146,-0.315275,-0.412497,-0.512733,-0.579654,-0.585606,-0.456732,0.165832,-0.000234,1.414382
1,0.715004,0.268140,-0.259919,-0.523365,-0.669745,-0.766248,-0.840503,-0.885586,-0.917978,-1.048123,...,2.151982,1.949846,1.751520,1.513120,1.262434,1.127323,1.190347,0.165832,-1.224088,1.414382
2,-0.160492,-0.150986,-0.247582,-0.341902,-0.427747,-0.513614,-0.596431,-0.649407,-0.655274,-0.588250,...,-0.337044,-0.421592,-0.511753,-0.600784,-0.659492,-0.672546,-0.612427,-0.824542,-1.224088,1.414382
3,-0.050883,0.094143,0.250059,0.434183,0.632276,0.824081,0.939695,0.932325,0.875651,0.751999,...,0.412376,0.635428,0.848652,0.965064,0.946070,0.900340,0.778926,-0.164293,1.223620,-0.707815
4,1.595827,1.664102,1.691679,1.616506,1.527805,1.427961,1.276346,1.080011,0.940387,0.920082,...,-0.150790,-0.229983,-0.274726,-0.325982,-0.379540,-0.421430,-0.529112,0.661019,-0.000234,-0.707815


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,-0.679571,-0.668409,-0.603837,-0.507884,-0.383752,-0.229052,-0.029838,0.086457,0.124478,0.053194,...,0.378474,0.493017,0.621928,0.776988,0.826915,0.841023,0.834131,-1.649854,-0.000234,-0.707815
1,1.439153,1.470612,1.437357,1.353716,1.253217,1.130355,0.961936,0.775627,0.668507,0.765128,...,-1.308757,-1.360934,-1.386747,-1.391639,-1.363853,-1.344996,-1.413634,-1.154667,-1.224088,1.414382
2,-0.419754,-0.388688,-0.374172,-0.351033,-0.312608,-0.251658,-0.134293,0.062617,0.183179,0.216037,...,-0.350105,-0.305226,-0.241035,-0.117665,0.094264,0.225373,0.262056,0.330894,-1.224088,0.706983
3,1.026671,1.204903,1.260034,1.325985,1.421895,1.557278,1.773398,2.161190,2.448282,2.563510,...,-1.356756,-1.192833,-0.929988,-0.514793,0.119781,0.423694,0.266918,0.826082,-1.224088,-0.707815
4,0.035100,0.055657,-0.059504,-0.168853,-0.261716,-0.352264,-0.443812,-0.511440,-0.532319,-0.484888,...,-0.163686,-0.257133,-0.351937,-0.448344,-0.519238,-0.542896,-0.497865,-0.329355,1.223620,-0.707815


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,-0.692405,-0.592585,-0.686633,-0.834649,-0.988531,-1.112508,-1.177856,-1.161541,-1.081896,-0.872864,...,-1.725294,-1.856136,-1.962570,-1.995849,-1.935133,-1.846052,-1.707648,1.651394,-0.000234,-1.415214
1,1.547611,1.084243,1.015691,0.900559,0.750244,0.571352,0.375267,0.196998,0.106589,0.199075,...,-0.863962,-1.049968,-1.189648,-1.284991,-1.320928,-1.330226,-1.355719,-0.824542,-1.224088,-1.415214
2,0.276699,1.029724,1.367117,1.239094,0.672818,-0.057723,-0.732369,-1.234057,-1.520831,-1.636846,...,3.014230,2.331106,1.547437,0.785823,0.157837,-0.226843,-0.243340,0.495957,1.223620,1.414382
3,1.309604,1.567347,1.773714,1.921063,1.932388,1.703832,1.181365,0.445325,-0.190608,-0.446837,...,1.024128,1.042516,0.759319,0.233139,-0.395522,-0.934411,-1.257290,-0.659480,-1.224088,1.414382
4,-0.097396,-0.011687,-0.013506,-0.042041,-0.092803,-0.167455,-0.255821,-0.318752,-0.287176,0.086768,...,-0.927431,-0.954290,-0.996755,-1.035270,-1.031782,-0.963340,-0.624621,-1.319730,-0.000234,-0.000416


In [21]:
# displaying y_train, y_val & y_test after standardization
display(pd.DataFrame(y_train).head())
display(pd.DataFrame(y_val).head())
display(pd.DataFrame(y_test).head())

,CL,CD,Cm
5345,0.126,0.01597,-0.001
158960,1.001,0.00242,-0.097
38414,-0.399,0.02010,-0.023
153330,1.212,0.02784,-0.304
147245,1.724,0.04200,-0.161


,CL,CD,Cm
102186,-0.369,0.02171,-0.206
76947,-0.399,0.02949,-0.059
69375,0.919,0.02188,-0.168
123201,2.144,0.04763,-0.339
56309,0.116,0.02005,-0.044


,CL,CD,Cm
167495,0.862,0.03265,0.070
172100,-0.247,0.02068,-0.002
226267,1.127,0.02623,0.011
209355,0.419,0.02355,-0.087
235769,-0.932,0.02733,-0.016


In [22]:
# defining the learning rate reduction callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=5, min_lr=0.00001)

### Model [512, 256, 128, 3]

In [23]:
# RMSE and R² metric scores on test data for Model M_512_256_128
M_512_256_128_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_512_256_128...")

    M_512_256_128 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_512_256_128.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_512_256_128.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_512_256_128.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_512_256_128_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_512_256_128_df = pd.DataFrame(M_512_256_128_metrics)

print(results_M_512_256_128_df.mean())



Run 1/20 for Model M_512_256_128...


I0000 00:00:1764215614.466752 4013331 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6155 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
I0000 00:00:1764215617.520652 4013444 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.



Run 2/20 for Model M_512_256_128...

Run 3/20 for Model M_512_256_128...

Run 4/20 for Model M_512_256_128...

Run 5/20 for Model M_512_256_128...

Run 6/20 for Model M_512_256_128...

Run 7/20 for Model M_512_256_128...

Run 8/20 for Model M_512_256_128...

Run 9/20 for Model M_512_256_128...

Run 10/20 for Model M_512_256_128...

Run 11/20 for Model M_512_256_128...

Run 12/20 for Model M_512_256_128...

Run 13/20 for Model M_512_256_128...

Run 14/20 for Model M_512_256_128...

Run 15/20 for Model M_512_256_128...

Run 16/20 for Model M_512_256_128...

Run 17/20 for Model M_512_256_128...

Run 18/20 for Model M_512_256_128...

Run 19/20 for Model M_512_256_128...

Run 20/20 for Model M_512_256_128...
CL_rmse    0.061315
CD_rmse    0.008127
Cm_rmse    0.030858
CL_r2      0.995584
CD_r2      0.471389
Cm_r2      0.881492
dtype: float32
